In [1]:
#hide
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 84.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 98.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 77.2 MB/s eta 0:00:00
Mounted at /content/gdrive


In [2]:
#hide
from fastbook import *
from IPython.display import display,HTML

NLP Deep Dive: RNNs

Text Preprocessing

Tokenization

In [3]:
from fastai.text.all import *
path = untar_data(URLs.IMDB)

In [4]:
files = get_text_files(path, folders = ['train', 'test', 'unsup'])

In [5]:
txt = files[0].open().read(); txt[:75]

'This is a true story of an Australian couple wha are charged with murder wh'

In [6]:
spacy = WordTokenizer()
toks = first(spacy([txt]))
print(coll_repr(toks, 30))

(#216) ['This','is','a','true','story','of','an','Australian','couple','wha','are','charged','with','murder','when','their','infant','child','disappears','.','Meryl','Streep','is','excellent',',','as','always',',','and','manages'...]


In [7]:
first(spacy(['The U.S. dollar 1is1.00.']))

(#5) ['The','U.S.','dollar','1is1.00','.']

Unable to replicate the font. Probably in Latex.

Word Tokenization with fastai

In [8]:
tkn = Tokenizer(spacy)
print(coll_repr(tkn(txt), 31))

(#250) ['xxbos','xxmaj','this','is','a','true','story','of','an','xxmaj','australian','couple','wha','are','charged','with','murder','when','their','infant','child','disappears','.','xxmaj','meryl','xxmaj','streep','is','excellent',',','as'...]


In [9]:
defaults.text_proc_rules

[<function fastai.text.core.fix_html(x)>,
 <function fastai.text.core.replace_rep(t)>,
 <function fastai.text.core.replace_wrep(t)>,
 <function fastai.text.core.spec_add_spaces(t)>,
 <function fastai.text.core.rm_useless_spaces(t)>,
 <function fastai.text.core.replace_all_caps(t)>,
 <function fastai.text.core.replace_maj(t)>,
 <function fastai.text.core.lowercase(t, add_bos=True, add_eos=False)>]

In [10]:
coll_repr(tkn('©   Fast.ai www.fast.ai/INDEX'), 31)

"(#11) ['xxbos','©','xxmaj','fast.ai','xxrep','3','w','.fast.ai','/','xxup','index']"

Subword Tokenization

In [11]:
txts = L(o.open().read() for o in files[:2000])

In [12]:
def subword(sz):
    sp = SubwordTokenizer(vocab_sz=sz)
    sp.setup(txts)
    return ' '.join(first(sp([txt]))[:40])

In [13]:
subword(1000)

'▁This ▁is ▁a ▁true ▁story ▁of ▁an ▁A us t ra li an ▁couple ▁w ha ▁are ▁ch ar g ed ▁with ▁murder ▁when ▁their ▁in f ant ▁child ▁dis app e ar s . ▁M er y l ▁St'

In [14]:
subword(200)

'▁ T h i s ▁is ▁a ▁ t r u e ▁ st or y ▁of ▁ an ▁A u st ra li an ▁c o u p le ▁w ha ▁a re ▁ ch ar g ed ▁with'

In [15]:
subword(10000)

'▁This ▁is ▁a ▁true ▁story ▁of ▁an ▁Australian ▁couple ▁wha ▁are ▁charge d ▁with ▁murder ▁when ▁their ▁infant ▁child ▁disappear s . ▁Meryl ▁Streep ▁is ▁excellent , ▁as ▁always , ▁and ▁manage s ▁to ▁hold ▁our ▁interest ▁even ▁though ▁she'

Numericalization

In [16]:
toks = tkn(txt)
print(coll_repr(tkn(txt), 31))

(#250) ['xxbos','xxmaj','this','is','a','true','story','of','an','xxmaj','australian','couple','wha','are','charged','with','murder','when','their','infant','child','disappears','.','xxmaj','meryl','xxmaj','streep','is','excellent',',','as'...]


In [17]:
toks200 = txts[:200].map(tkn)
toks200[0]

(#250) ['xxbos','xxmaj','this','is','a','true','story','of','an','xxmaj'...]

In [18]:
num = Numericalize()
num.setup(toks200)
coll_repr(num.vocab,20)

"(#2184) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the',',','.','and','a','of','to','is','in','it','i'...]"

In [19]:
nums = num(toks)[:20]; nums

TensorText([   2,    8,   22,   16,   13,  294,   62,   14,   50,    8,    0,  350,    0,   37, 1568,   30,  846,   63,   83,    0])

In [20]:
' '.join(num.vocab[o] for o in nums)

'xxbos xxmaj this is a true story of an xxmaj xxunk couple xxunk are charged with murder when their xxunk'

Putting Our Texts into Batches for a Language Model

In [21]:
#hide_input
stream = "In this chapter, we will go back over the example of classifying movie reviews we studied in chapter 1 and dig deeper under the surface. First we will look at the processing steps necessary to convert text into numbers and how to customize it. By doing this, we'll have another example of the PreProcessor used in the data block API.\nThen we will study how we build a language model and train it for a while."
tokens = tkn(stream)
bs,seq_len = 6,15
d_tokens = np.array([tokens[i*seq_len:(i+1)*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

xxbos,xxmaj,in,this,chapter,",",we,will,go,back,over,the,example,of,classifying
movie,reviews,we,studied,in,chapter,1,and,dig,deeper,under,the,surface,.,xxmaj
first,we,will,look,at,the,processing,steps,necessary,to,convert,text,into,numbers,and
how,to,customize,it,.,xxmaj,by,doing,this,",",we,'ll,have,another,example
of,the,preprocessor,used,in,the,data,block,xxup,api,.,\n,xxmaj,then,we
will,study,how,we,build,a,language,model,and,train,it,for,a,while,.


In [22]:
#hide_input
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15:i*15+seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

xxbos,xxmaj,in,this,chapter
movie,reviews,we,studied,in
first,we,will,look,at
how,to,customize,it,.
of,the,preprocessor,used,in
will,study,how,we,build


In [23]:
#hide_input
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+seq_len:i*15+2*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

",",we,will,go,back
chapter,1,and,dig,deeper
the,processing,steps,necessary,to
xxmaj,by,doing,this,","
the,data,block,xxup,api
a,language,model,and,train


In [24]:
#hide_input
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+10:i*15+15] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

over,the,example,of,classifying
under,the,surface,.,xxmaj
convert,text,into,numbers,and
we,'ll,have,another,example
.,\n,xxmaj,then,we
it,for,a,while,.


In [25]:
nums200 = toks200.map(num)

In [26]:
dl = LMDataLoader(nums200)

In [27]:
x,y = first(dl)
x.shape,y.shape

(torch.Size([64, 72]), torch.Size([64, 72]))

In [28]:
' '.join(num.vocab[o] for o in x[0][:20])

'xxbos xxmaj this is a true story of an xxmaj xxunk couple xxunk are charged with murder when their xxunk'

In [29]:
' '.join(num.vocab[o] for o in y[0][:20])

'xxmaj this is a true story of an xxmaj xxunk couple xxunk are charged with murder when their xxunk child'

Training a Text Classifier

In [30]:
get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])

dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_imdb, splitter=RandomSplitter(0.1)
).dataloaders(path, path=path, bs=128, seq_len=80)


In [31]:
dls_lm.show_batch(max_n=2)

,text,text_
0,"xxbos xxmaj the main word that comes to mind when considering this film is "" dodgy "" . xxmaj this is a low - quality film biography of one of the most iconic performers of all time . xxmaj the xxmaj gloved xxmaj one deserved better . \n\n xxmaj before getting into the meat of my thoughts on this biopic , i have to say that there are two things i found effective . xxmaj first was the use of","xxmaj the main word that comes to mind when considering this film is "" dodgy "" . xxmaj this is a low - quality film biography of one of the most iconic performers of all time . xxmaj the xxmaj gloved xxmaj one deserved better . \n\n xxmaj before getting into the meat of my thoughts on this biopic , i have to say that there are two things i found effective . xxmaj first was the use of actual"
1,"is and , with the usual lack of emotion , shoots him as he asks her to marry him ; my only thought was , why would this guy want to marry this freakish zombie - lady ? xxmaj at this point , the film lost all credibility , ha - ha . xxmaj also , to get any enjoyment out of this movie , you have to be the type that really appreciates overweight women who keep swinging grotesquely","and , with the usual lack of emotion , shoots him as he asks her to marry him ; my only thought was , why would this guy want to marry this freakish zombie - lady ? xxmaj at this point , the film lost all credibility , ha - ha . xxmaj also , to get any enjoyment out of this movie , you have to be the type that really appreciates overweight women who keep swinging grotesquely xxunk"


Fine-Tuning the Language Model

In [32]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3,
    metrics=[accuracy, Perplexity()]).to_fp16()

In [ ]:
learn.fit_one_cycle(1, 2e-2)

/usr/local/lib/python3.10/dist-packages/torch/amp/autocast_mode.py:204: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/usr/local/lib/python3.10/dist-packages/torch/cuda/amp/grad_scaler.py:120: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


epoch,train_loss,valid_loss,accuracy,perplexity,time


In [ ]:
learn.save('1epoch')

In [ ]:
learn = learn.load('1epoch')

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, 2e-3)

In [ ]:
learn.save_encoder('finetuned')